In [7]:
import sys
import re
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs
from datetime import datetime, date
from datetime import timedelta

In [8]:
teams = pd.read_csv('/Users/Kevin/Desktop/fanduel/teams.csv')

In [15]:
def find_date(table_row,year=2019):    
    date_str = re.findall(r'[A-Z][a-z][a-z],\s[A-Z][a-z][a-z] \d+',str(table_row))[0]
    if 'Feb 29' in date_str:
        return datetime.strptime(date_str + str(year-1),'%a, %b %d%Y')
    if year < 2020:
        if datetime.strptime(date_str,'%a, %b %d').month > 8:
            return datetime.strptime(date_str,'%a, %b %d').replace(year = year-1)
        else:            
            return datetime.strptime(date_str,'%a, %b %d').replace(year = year)
    if year == 2020:
        if datetime.strptime(date_str,'%a, %b %d').month > 9:
            return datetime.strptime(date_str,'%a, %b %d').replace(year = year-1)
        else:            
            return datetime.strptime(date_str,'%a, %b %d').replace(year = year)
    if year == 2021:
        if datetime.strptime(date_str,'%a, %b %d').month > 11:
            return datetime.strptime(date_str,'%a, %b %d').replace(year = year-1)
        else:            
            return datetime.strptime(date_str,'%a, %b %d').replace(year = year)    
    
    
def get_text(short_name,year):
    BASE_URL = f'https://www.espn.com/nba/team/schedule/_/name/{short_name}/season/{year}/seasontype/2'
    print(BASE_URL)
    r = requests.get(BASE_URL)
    text = bs(r.text) 
    return text

def get_table_rows(text):
    return text.find_all('tr')

def get_anchor_tags(table_row):
    return table_row.find_all('a')

def get_id_score_date_record(table_row, anchor_tag,year = 2019):
    if 'gameId' in anchor_tag['href']:
        game_id = anchor_tag['href'].split('gameId=')[1]
        scores = anchor_tag.text.strip()
        date = find_date(table_row, year)
        record = [r for r in re.findall(r'[0-9]+\-[0-9]+',str(table_row))][1]
        return game_id, scores, date, record

    
def get_opponent(anchor_tag):
    if 'nba/team/' in anchor_tag['href'] and 'teamlogos' not in str(anchor_tag):
        team_long_short = anchor_tag['href'].split('/')[-2:]
        other_team = team_long_short[-2]
        return other_team

    
def home_away(table_row, current_team, other_team):
    '''decide whether current team is home or away'''
    if 'vs' in [i.text for i in table_row.find_all(class_ = 'pr2')]:
        home_team = current_team
        visit_team = other_team
    else:
        home_team = other_team
        visit_team = home_team
        
    return home_team,visit_team


def get_record(table_row):
    return [r for r in re.findall(r'[0-9]+\-[0-9]+',str(i))][1]


def update_records(home_team, current_team, game_id, past_record, all_records, new_record):
    '''Update records dictionary'''
    # Having trouble with this function updateing home and away
    #print(home_team, current_team, game_id, past_record, all_records, new_record)
    if game_id not in all_records.keys():
        all_records[game_id] = {}
    if home_team == current_team:
        all_records[game_id]['home_record'] = past_record
    else:
        all_records[game_id]['away_record'] = past_record
    past_record = new_record
    return all_records, past_record
    

#def only_get_href_anchors(anchor_tags):
#    for i in anchor_tags
        

In [16]:
#most complete thus far, gets score, W vs L and boxscore link, date
#date	home_team	visit_team	home_team_score	visit_team_score	away_losses	away_wins	home_losses	home_wins

all_records = {}
full_df = pd.DataFrame()
current_year = 2021

for current_team in teams.prefix_1:
    #if current_team == 'det':
    
    BASE_URL = f'https://www.espn.com/nba/team/schedule/_/name/{current_team}/season/{current_year}/seasontype/2'
    r = requests.get(BASE_URL)
    text = bs(r.text)
    

    match_id = []
    dates = []
    home_teams = []
    home_team_scores = []
    visit_teams = []
    visit_team_scores = []
    b=0
    allcolumns = []
    past_record = '0-0'
    game_ids = []
    # find all table rows
    for i in text.find_all("tr"):
        # why do we use try? I believe it's because either some don't have anchor tags or they don't have href
        # Want to create a wrapper function to skip over these
        got_here = False
        
        try:
            for j in i.find_all('a'):
                if 'gameId' in j['href']:
                    got_here = True
                    game_id = j['href'].split('gameId=')[1]

                    scores = j.text.strip()
                    
                    date = find_date(i,year=current_year)
                    record = [r for r in re.findall(r'[0-9]+\-[0-9]+',str(i))][1]
                    #print('date',date)
                    page = requests.get(j['href'])
                    for k in bs(page.text).find_all('a',class_='view-more webview-internal'):
                        if 'boxscore' in k['href']:
                            box_score = 'www.espn.com'+k['href']
                else:
                    pass
                if 'nba/team/' in j['href'] and 'teamlogos' not in str(j):
                    team_long_short = j['href'].split('/')[-2:]
                    short_name = team_long_short[-2]
                    #print('opponent',short_name)
                else:
                    pass

                #print([home_away.text for home_away in i.find_all(class_ = 'pr2')])
            if 'vs' in [i.text for i in i.find_all(class_ = 'pr2')]:
                home_team = current_team
                if game_id not in all_records.keys():
                    all_records[game_id] = {}
                all_records[game_id]['home_record'] = past_record
                visit_team = short_name
                #print(record,past_record)

            else:
                home_team = short_name
                visit_team = current_team
                if game_id not in all_records.keys():
                    all_records[game_id] = {}
                all_records[game_id]['away_record'] = past_record
                #print(record,past_record)




            for l in i.find_all(class_ = "fw-bold clr-negative"):
                #print("LOSS",l.text)
                win_loss = l.text
            for w in i.find_all(class_ = "fw-bold clr-positive"):
                #print("WIN",w.text)  
                win_loss = w.text
            #print(win_loss)

            scores_list = re.findall(r'\d+',scores)
            #print('scores',scores_list)
            if win_loss == 'L':
                winner = short_name
            else:
                winner = current_team

            if home_team == winner:
                home_team_score = scores_list[0]
                visit_team_score = scores_list[1]
            else:
                home_team_score = scores_list[1]
                visit_team_score = scores_list[0]


            if got_here == True:

                dates.append(date)
                home_teams.append(home_team)
                visit_teams.append(visit_team)
                home_team_scores.append(home_team_score)
                visit_team_scores.append(visit_team_score)
                game_ids.append(game_id)

                print('looping over:',current_team)
                print('past: ',past_record)
                print('current: ',record)

                past_record = record

                print('home_team',home_team)
                print('away_team',visit_team)
                print('game_id:',game_id)
                #print('home_team_score: ',home_team_score)
                #print('visit_team_score',visit_team_score)
                print('next\n')





        except Exception as e:
            print(e)
            pass


    df = pd.DataFrame([game_ids,dates,home_teams,visit_teams, home_team_scores, visit_team_scores]).T
    df.columns = ['game_ids','date','home_team','visit_team','home_team_score','visit_team_score']
    full_df = full_df.append(df)
#else:
#    pass



looping over: bos
past:  0-0
current:  1-0
home_team bos
away_team mil
game_id: 401266804
next

looping over: bos
past:  1-0
current:  1-1
home_team bos
away_team bkn
game_id: 401266798
next

looping over: bos
past:  1-1
current:  1-2
home_team ind
away_team bos
game_id: 401267197
next

looping over: bos
past:  1-2
current:  2-2
home_team ind
away_team bos
game_id: 401267208
next

looping over: bos
past:  2-2
current:  3-2
home_team bos
away_team mem
game_id: 401267216
next

looping over: bos
past:  3-2
current:  3-3
home_team det
away_team bos
game_id: 401267230
next

looping over: bos
past:  3-3
current:  4-3
home_team det
away_team bos
game_id: 401267245
next

looping over: bos
past:  4-3
current:  5-3
home_team tor
away_team bos
game_id: 401267257
next

looping over: bos
past:  5-3
current:  6-3
home_team mia
away_team bos
game_id: 401267271
next

list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out o

looping over: cle
past:  0-0
current:  1-0
home_team cle
away_team cha
game_id: 401267150
next

looping over: cle
past:  1-0
current:  2-0
home_team det
away_team cle
game_id: 401267183
next

looping over: cle
past:  2-0
current:  3-0
home_team cle
away_team phi
game_id: 401267195
next

looping over: cle
past:  3-0
current:  3-1
home_team cle
away_team ny
game_id: 401267206
next

looping over: cle
past:  3-1
current:  3-2
home_team ind
away_team cle
game_id: 401267222
next

looping over: cle
past:  3-2
current:  4-2
home_team atl
away_team cle
game_id: 401267243
next

looping over: cle
past:  4-2
current:  4-3
home_team orl
away_team cle
game_id: 401267253
next

looping over: cle
past:  4-3
current:  4-4
home_team orl
away_team cle
game_id: 401267268
next

looping over: cle
past:  4-4
current:  5-4
home_team mem
away_team cle
game_id: 401267279
next

list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of

looping over: okc
past:  0-0
current:  1-0
home_team cha
away_team okc
game_id: 401267182
next

looping over: okc
past:  1-0
current:  1-1
home_team okc
away_team utah
game_id: 401267203
next

looping over: okc
past:  1-1
current:  1-2
home_team okc
away_team orl
game_id: 401267212
next

looping over: okc
past:  1-2
current:  1-3
home_team okc
away_team no
game_id: 401267227
next

looping over: okc
past:  1-3
current:  2-3
home_team orl
away_team okc
game_id: 401267241
next

looping over: okc
past:  2-3
current:  2-4
home_team mia
away_team okc
game_id: 401267256
next

looping over: okc
past:  2-4
current:  3-4
home_team no
away_team okc
game_id: 401267274
next

list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of

looping over: phx
past:  0-0
current:  1-0
home_team phx
away_team dal
game_id: 401266796
next

looping over: phx
past:  1-0
current:  1-1
home_team sac
away_team phx
game_id: 401267190
next

looping over: phx
past:  1-1
current:  2-1
home_team sac
away_team phx
game_id: 401267199
next

looping over: phx
past:  2-1
current:  3-1
home_team phx
away_team no
game_id: 401267213
next

looping over: phx
past:  3-1
current:  4-1
home_team utah
away_team phx
game_id: 401267228
next

looping over: phx
past:  4-1
current:  5-1
home_team den
away_team phx
game_id: 401267236
next

looping over: phx
past:  5-1
current:  5-2
home_team phx
away_team lac
game_id: 401267251
next

looping over: phx
past:  5-2
current:  6-2
home_team phx
away_team tor
game_id: 401267275
next

list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out o

looping over: wsh
past:  0-0
current:  0-1
home_team phi
away_team wsh
game_id: 401267173
next

looping over: wsh
past:  0-1
current:  0-2
home_team wsh
away_team orl
game_id: 401267184
next

looping over: wsh
past:  0-2
current:  0-3
home_team wsh
away_team orl
game_id: 401267193
next

looping over: wsh
past:  0-3
current:  0-4
home_team wsh
away_team chi
game_id: 401267210
next

looping over: wsh
past:  0-4
current:  0-5
home_team wsh
away_team chi
game_id: 401267223
next

looping over: wsh
past:  0-5
current:  1-5
home_team min
away_team wsh
game_id: 401267234
next

looping over: wsh
past:  1-5
current:  2-5
home_team bkn
away_team wsh
game_id: 401267246
next

looping over: wsh
past:  2-5
current:  2-6
home_team phi
away_team wsh
game_id: 401267269
next

list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out o

In [14]:
full_df

,game_ids,date,home_team,visit_team,home_team_score,visit_team_score
0,401266804,2021-12-23,bos,mil,122,121
1,401266798,2021-12-25,bos,bkn,95,123
2,401267197,2021-12-27,ind,bos,108,107
3,401267208,2021-12-29,ind,bos,111,116
4,401267216,2021-12-30,bos,mem,126,107
...,...,...,...,...,...,...
3,401267220,2021-12-30,sa,lal,107,121
4,401267235,2021-01-01,sa,lal,103,109
5,401267249,2021-01-03,sa,utah,109,130
6,401267265,2021-01-05,lac,sa,113,116


In [14]:
full_df.drop_duplicates()

,game_ids,date,home_team,visit_team,home_team_score,visit_team_score
0,400899380,2016-10-26,bos,bkn,122,117
1,400899422,2016-10-27,chi,bos,105,99
2,400899477,2016-10-29,cha,bos,98,104
3,400899511,2016-11-02,bos,chi,107,100
4,400899518,2016-11-03,cle,bos,128,122
...,...,...,...,...,...,...
2,400899482,2016-10-29,sa,no,98,79
28,400899854,2016-12-18,sa,no,113,100
46,400900122,2017-01-27,no,sa,119,103
61,400900312,2017-03-03,no,sa,98,101


In [15]:
import pickle

In [16]:
current_year

2017

In [17]:
pickle.dump(full_df.drop_duplicates(),open(f'full_df_{current_year}.p','wb'))

In [18]:
pickle.dump(all_records,open(f'all_records_{current_year}.p','wb'))

In [ ]:
#getting links to each boxscore
for i in text.find_all('a',class_ = 'AnchorLink', tabindex ="0"):
    if 'gameId' in i['href']:
        page = requests.get(i['href'])
        for i in bs(page.text).find_all('a',class_='view-more webview-internal'):
            if 'boxscore' in i['href']:
                print('www.espn.com'+i['href'])


In [ ]:
for i in text.find_all('a',class_ = 'AnchorLink', tabindex ="0"):
    if 'gameId' in i['href']:
        print(i.text)
        print(i)
        break

In [ ]:
page = requests.get('http://www.espn.com/nba/game?gameId=401224783')
for i in bs(page.text).find_all('a',class_='view-more webview-internal'):
    if 'boxscore' in i['href']:
        print('www.espn.com'+i['href'],'\n')

In [ ]:
for row in table.find_all('tr')[2:3]:
    columns = row.find_all('td')
    print(bs.prettify(columns[2]))

In [ ]:


year = int(input("year?: "))
#print("hhhhh"+year+"hhhhh")
teams = pd.read_csv('/Users/Kevin/Desktop/fanduel/teams.csv')
BASE_URL = 'http://espn.go.com/nba/team/schedule/_/name/{0}/year/{1}/seasontype/2'

match_id = []
dates = []
home_team = []
home_team_score = []
visit_team = []
visit_team_score = []
b=0
allcolumns = []
allrecords = []
for index, row in teams.iterrows():
    _team, url = row['team'], row['url']
    r = requests.get(BASE_URL.format(row['prefix_1'], year, row['prefix_2']))
    table = BeautifulSoup(r.text,'lxml').table
    try:
        for row in table.find_all('tr')[1:]: # Remove header

            columns = row.find_all('td')
            try:
   
                _home = True if columns[1].li.text == 'vs' else False

                _other_team = columns[1].find_all('a')[1].text

                _score = columns[2].a.text.split(' ')[0].split('-')
                _won = True if columns[2].span.text == 'W' else False
                a = re.sub('<td>','',str(columns[3]))
                b = re.sub('</td>','',a)
                _home_record = b if _home else None
                _away_record = b if not _home else 0


                match_id.append(columns[2].a['href'].split('id/')[1])
                home_team.append(_team if _home else _other_team)
                visit_team.append(_team if not _home else _other_team)
                d = datetime.strptime(columns[0].text, '%a, %b %d')
                if d.month > 8:
                    dates1 = date(year-1,d.month,d.day)
                    dates.append(date(year-1,d.month,d.day))
                else:
                    dates.append(date(year, d.month, d.day))
                    dates1 = date(year, d.month, d.day)
                a = re.sub('<td>','',str(columns[3]))
                b = re.sub('</td>','',a)
                c = b.split('-')
                _home_wins = c[0] if _home else 0
                _home_losses = c[1] if _home else 0
                _away_wins = c[0] if not _home else 0
                _away_losses = c[1] if not _home else 0
                allrecords.append([columns[2].a['href'].split('id/')[1],_home_wins,_home_losses,_away_wins,_away_losses]) 
                

                if _home:
                    if _won:
                        home_score=_score[0]
                        visit_score=_score[1]
                        home_team_score.append(_score[0])
                        visit_team_score.append(_score[1])
                    else:
                        home_score=_score[1]
                        visit_score=_score[0]
                        home_team_score.append(_score[1])
                        visit_team_score.append(_score[0])

                else:
                    if _won:
                        home_score=_score[1]
                        visit_score=_score[0]                    
                        home_team_score.append(_score[1])
                        visit_team_score.append(_score[0])
                    else:
                        home_score=_score[0]
                        visit_score=_score[1]                    
                        home_team_score.append(_score[0])
                        visit_team_score.append(_score[1])

                allcolumns.append([columns[2].a['href'].split('id/')[1],
                                   dates1,
                                   _team if _home else _other_team,
                                   _team if not _home else _other_team,
                                 home_score,visit_score])

            except Exception as e:

                pass # Not all columns row are a match, is OK
                    # print(e)
    except Exception as e:
        pass


dic = {'id': match_id, 'date': dates, 'home_team': home_team, 'visit_team': visit_team,'home_team_score': home_team_score, 'visit_team_score': visit_team_score}
headers = ['id','date','home_team','visit_team','home_team_score','visit_team_score']
games = pd.DataFrame(allcolumns,columns=headers)
recordheaders = ['id','home_wins','home_losses','away_wins','away_losses']
recordtable = pd.DataFrame(allrecords,columns=recordheaders)


#Create pivot table to correctly identify win and loss records of teams
temp = recordtable.fillna(0)

temp.home_wins = temp.home_wins.apply(int)
temp.home_losses = temp.home_losses.apply(int)
temp.away_wins = temp.away_wins.apply(int)
temp.away_losses = temp.away_losses.apply(int)
recdf = temp.pivot_table(index='id',aggfunc=np.sum)

recdf.to_csv('records'+str(year)+'.csv')
recdf.tail()


games1 = games.drop_duplicates(subset = 'id',keep = 'last').set_index('id')

city_team = ['Atlanta', 'Boston', 'Brooklyn', 'Charlotte', 'Chicago',
       'Cleveland', 'Dallas', 'Denver', 'Detroit','Golden State', 'Houston', 'Indiana',
       'LA','Los Angeles', 'Memphis', 'Miami', 'Milwaukee', 'Minnesota','New Orleans','NY Knicks',
       'Oklahoma City','Orlando','Philadelphia', 'Phoenix', 'Portland', 'Sacramento','San Antonio', 'Toronto',
       'Utah','Washington','Atlanta Hawks', 'Boston Celtics', 'Brooklyn Nets',
       'Charlotte Hornets', 'Chicago Bulls', 'Cleveland Cavaliers',
       'Dallas Mavericks', 'Denver Nuggets', 'Detroit Pistons',
       'Golden State Warriors', 'Houston Rockets','Indiana Pacers',
       'LA Clippers', 'Los Angeles Lakers',
       'Memphis Grizzlies', 'Miami Heat', 'Milwaukee Bucks',
       'Minnesota Timberwolves',
       'New Orleans Pelicans', 'New York Knicks',
       'Oklahoma City Thunder', 'Orlando Magic', 'Philadelphia 76ers',
       'Phoenix Suns', 'Portland Trail Blazers', 'Sacramento Kings',
       'San Antonio Spurs', 'Toronto Raptors', 'Utah Jazz',
       'Washington Wizards']

final = ['ATL', 'BOS','BKN', 'CHA', 'CHI', 'CLE', 'DAL', 'DEN', 'DET',
       'GS', 'HOU', 'IND', 'LAC', 'LAL', 'MEM', 'MIA', 'MIL', 'MIN', 'NO',
       'NY', 'OKC', 'ORL', 'PHI', 'PHX', 'POR', 'SAC', 'SA', 'TOR', 'UTA',
       'WSH']*2

team_dict = {k:v for k,v in zip(city_team,final)}

games1['visit_team'] = games1.visit_team.map(team_dict)

games1['home_team'] = games1.home_team.map(team_dict)

games1 = games1.join(recdf)

games1.to_csv('/Users/Kevin/Desktop/fanduel/games'+str(year)+'.csv')

#conn = psycopg2.connect(database="postgres", user="postgres", password="pass", host="localhost", port="5432")

#cur = conn.cursor()

#cur.execute('DROP TABLE IF EXISTS games%s;' %year)
#cur.execute('CREATE TABLE games%s (id varchar(50),date varchar(50),home_team varchar(50),visit_team varchar(50),home_team_score int,visit_team_score int,away_losses int,away_wins int,home_losses int,home_wins int)' %year)

#cur.execute("COPY games%s FROM '/home/w205/final_project/csvfiles/games%s.csv' DELIMITER ',' CSV HEADER;" %(year,year)) 

#conn.commit()

# Eventually moving everything to the functions above to clean up

In [ ]:
'''
all_records = {}
current_team = 'bos'
past_record = '0-0'
game_id = ''
text = get_text(current_team)
trs = get_table_rows(text)
for table_row in trs:
    a_ts = get_anchor_tags(table_row)
    for anchor_tag in a_ts:
        try:
            if 'gameId' in anchor_tag['href']:
                game_id, score, date, new_record = get_id_score_date_record(table_row,anchor_tag,year = 2019) 
                #print(game_id)
            other_team = get_opponent(anchor_tag)
            #print(all_records)
            home_team, visit_team = home_away(table_row, current_team, other_team)
            all_records, past_record = update_records(home_team,
                                                      current_team,
                                                      game_id,
                                                      past_record,
                                                      all_records,
                                                      new_record)
            #print(all_records)
            
        except:
            pass'''